In [3]:
a=[]

In [4]:
a.append(1)

In [9]:
a

[1, 2, '2']

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import LinearRegression

In [2]:
eventtype=['accidentsAndIncidents','roadwork','precipitation','deviceStatus','obstruction','trafficConditions']
date_format="%Y-%m-%d"

def deltaday(date1, date2):
	time1=datetime.strptime(date1,date_format)
	time2=datetime.strptime(date2,date_format)
	return (time2-time1).days

In [3]:
df=pd.read_csv("/Users/LiuLuoming/Desktop/IDS/lab11_data/events_train.tsv",sep='\t',na_values=['-'])
pred=pd.read_csv("/Users/LiuLuoming/Desktop/IDS/lab11_data/prediction_trials.tsv",sep='\t',na_values=['-'])
df=df[['closed_tstamp','event_type','latitude','longitude']]

In [4]:
df['closed_tstamp']=df['closed_tstamp'].str.extract('(....-..-..)', expand=True)
df['closed_tstamp']=df['closed_tstamp'].map(lambda x: str(x)[:4])

In [5]:
pred['start']=pred['start'].str.extract('(....-..-..)', expand=True)
pred['end']=pred['end'].str.extract('(....-..-..)', expand=True)

In [6]:
del pred['trial_id']

In [8]:
pred_arr=pred[:].values #get the array represent of predication file

In [7]:
pred=pred[:1000]

In [9]:
res=[[0 for x in range(6)] for y in range(len(pred_arr))] # array used for store of the result 6 column

In [10]:

for index in range(len(pred_arr)):
    row=pred_arr[index]
    event=df[(df['latitude']>=min(row[0],row[2]))&
         (df['latitude']<=max(row[0],row[2]))&
         (df['longitude']>=min(row[1],row[3]))&
         (df['longitude']<=max(row[3],row[1]))]
    group=event.groupby('event_type')
    interval=deltaday(row[4],row[5])
    for i in range(6):
        hashset=set(group.groups.keys())
        if(eventtype[i] not in hashset):
            continue
        count=group.get_group(eventtype[i]).groupby('closed_tstamp').size()
        if(len(count)<2):
            continue
        x=[[x] for x in count[:len(count)-1]]
        y=count[1:]
        lm=LinearRegression()
        lm.fit(x,y)
        gap=int(str(row[4])[:4])-int(group.get_group(eventtype[i])['closed_tstamp'].max())
        prediction=y[len(y)-1]
        for j in range(gap):
            prediction=lm.predict(prediction)[0]
        res[index][i]=max(0,int(prediction*interval/365))

/usr/local/lib/python2.7/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [11]:
res

[[21, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0],
 [21, 0, 0, 0, 0, 0],
 [49, 0, 0, 0, 4, 0],
 [12, 2, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0],
 [12, 0, 0, 1, 0, 0],
 [53, 7, 0, 0, 0, 10],
 [115, 11, 0, 0, 8, 0],
 [2, 0, 0, 0, 0, 0],
 [43, 8, 0, 0, 0, 2],
 [6, 0, 0, 0, 0, 0],
 [0, 74, 0, 0, 0, 0],
 [4, 0, 0, 0, 0, 1],
 [19, 8, 0, 0, 0, 0],
 [23, 58, 0, 0, 2, 0],
 [1, 3, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [22, 9, 0, 0, 3, 0],
 [9, 0, 0, 0, 1, 0],
 [24, 1, 0, 0, 0, 2],
 [0, 0, 0, 0, 0, 0],
 [65, 0, 0, 0, 3, 0],
 [9, 0, 0, 2, 0, 0],
 [44, 0, 0, 0, 3, 0],
 [14, 1, 0, 0, 0, 4],
 [19, 3, 0, 0, 0, 2],
 [0, 0, 0, 0, 0, 0],
 [70, 5, 0, 0, 0, 5],
 [56, 8, 0, 0, 2, 0],
 [34, 23, 0, 0, 3, 0],
 [21, 2, 0, 0, 3, 0],
 [20, 0, 0, 0, 1, 0],
 [39, 36, 0, 0, 3, 1],
 [2, 1, 0, 0, 0, 0],
 [26, 4, 0, 0, 0, 0],
 [15, 0, 0, 0, 0, 0],
 [14, 0, 0, 0, 0, 0],
 [16, 0, 0, 0, 0, 0],
 [56, 10, 0, 0, 3, 0],
 [23, 4, 0, 0, 0, 19],
 [7, 76, 0, 0, 0, 0],
 [0, 58, 0, 0, 0, 0],
 [16, 1, 0, 0, 2, 0],
 [10, 0, 0, 0, 0, 0],
 [50, 8, 0, 0,

In [13]:
def predicateEvent(partData):
    pred_arr=partData[:].values
    lens=len(pred_arr)
    res=[[0 for x in range(6)] for y in range(len(pred_arr))]
    for index in range(lens):
        row=pred_arr[index]
        event=df[(df['latitude']>=min(row[0],row[2]))&
             (df['latitude']<=max(row[0],row[2]))&
             (df['longitude']>=min(row[1],row[3]))&
             (df['longitude']<=max(row[3],row[1]))]
        group=event.groupby('event_type')
        interval=deltaday(row[4],row[5])
        for i in range(6):
            hashset=set(group.groups.keys())
            if(eventtype[i] not in hashset):
                continue
            count=group.get_group(eventtype[i]).groupby('closed_tstamp').size()
            if(len(count)<2):
                continue
            x=[[x] for x in count[:len(count)-1]]
            y=count[1:]
            lm=LinearRegression()
            lm.fit(x,y)
            gap=int(str(row[4])[:4])-int(group.get_group(eventtype[i])['closed_tstamp'].max())
            prediction=y[len(y)-1]
            for j in range(gap):
                prediction=lm.predict(prediction)[0]
            res[index][i]=max(0,int(prediction*interval/365))
    res_dataframe= pd.DataFrame({'A':res[:,0],'R':res[:,1],'P':res[:,2],'D':res[:,3],'O':res[:,4],'T':res[:,5]})
    res_dataframe['id']=partData['trial_id']
    return res_dataframe
    

In [ ]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

parallelize_dataframe(pred, predicateEvent)